In [1]:
from pytrends.request import TrendReq
import pandas as pd
from datetime import datetime

In [2]:
pytrends = TrendReq(hl='en-US', tz=0) # 60 = CET 0 = UTC

# specifies current time as end date of results

now = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
now_d = int(now[:2])
now_m = int(now[3:5])
now_y = int(now[6:10])
now_h = int(now[11:13])

In [3]:
kw_list = ['Dogecoin', 'Samoyedcoin']
coin_list = []
data = []

#data now
for kw in kw_list:
    coin_list = [kw]
    data.append(pytrends.get_historical_interest(
            coin_list, year_start=now_y, month_start=10, day_start=4,
            hour_start=18, year_end=now_y, month_end=now_m, day_end=now_d,
            hour_end=now_h, cat=0, sleep=0))
data

[                     Dogecoin isPartial
 date                                   
 2021-07-31 00:00:00       100     False
 2021-07-31 01:00:00        96     False
 2021-07-31 02:00:00        96     False
 2021-07-31 03:00:00        92     False
 2021-07-31 04:00:00        92     False
 ...                       ...       ...
 2021-11-29 12:00:00        58     False
 2021-11-29 13:00:00       100     False
 2021-11-29 14:00:00        91     False
 2021-11-29 15:00:00        92     False
 2021-11-29 16:00:00        78      True
 
 [1586 rows x 2 columns],
                      Samoyedcoin isPartial
 date                                      
 2021-10-02 00:00:00            0     False
 2021-10-02 01:00:00            0     False
 2021-10-02 02:00:00            0     False
 2021-10-02 03:00:00            0     False
 2021-10-02 04:00:00            0     False
 ...                          ...       ...
 2021-11-29 12:00:00           35     False
 2021-11-29 13:00:00           59     False

In [5]:
df = data[0].iloc[:, 0]

    # merges all coins into one df
if len(kw_list) > 1:
    for i in range(1, len(kw_list)):
        df = pd.merge(left=df,
                        right=data[i].iloc[:, 0],
                        how='outer',
                        on='date')

# non-existing values are replaced by 0.25,
# (avg of 0 & 0.5, the range of values marked as <1 on GT)
df = df.fillna(0.25)

# set the index as to type datetime64[ns] to be ready for grouping
df.index = pd.to_datetime(df.index, format='%Y-%m-%d %H:%M:%S')
    
df

,Dogecoin,Samoyedcoin
date,,
2021-07-31 00:00:00,100,0.25
2021-07-31 01:00:00,96,0.25
2021-07-31 02:00:00,96,0.25
2021-07-31 03:00:00,92,0.25
2021-07-31 04:00:00,92,0.25
...,...,...
2021-11-29 12:00:00,58,35.00
2021-11-29 13:00:00,100,59.00
2021-11-29 14:00:00,91,21.00


In [6]:
df.to_csv('doge_samo_google_trends_3107-2911.csv')